# Convex Optimization - Homework 3 (Yui Chi HUNG)

## Problem 1: Deriving the dual problem of LASSO and formatting it as a general Quadratic Problem (QP)

Firstly, reformulate $\min_{w} \frac{1}{2}||Xw-y||_2^2+\lambda||w||_1$ as $\min_{w,z} \frac{1}{2}||z||_2^2+\lambda||w||_1$ with $Xw-z-y=0$.

Then, the Lagrangian is 
\begin{align*}
L(w,z,\nu) &=\frac{1}{2}||z||_2^2+\lambda||w||_1+\nu^T(Xw-z-y)\\
&=\frac{1}{2}||z||_2^2-\nu^T z+\nu^T Xw+\lambda||w||_1-\nu^Ty
\end{align*}

with $\nu\in\mathbb{R}^n$. Since the terms involving $w$ and $z$ are separable, we can minimize them separately when calculating the Lagrange dual function $g(\nu)=\inf_{w,z}L(w,z,\nu)$.

For $\inf_w \nu^T Xw+\lambda||w||_1$, if $||X^T\nu||_\infty > \lambda$, let $|(X^T\nu)_j|>\lambda$. By setting $w=-t\cdot\text{sign}((X^T\nu)_j)e_j$ with $e_j$ being the j-th basis vector in the canonical basis of $\mathbb{R}^d$, $\nu^T Xw+\lambda||w||_1=-t\lambda(|(X^T\nu)_j|-1)\to-\infty$ as $t\to\infty$. Hence we have $\inf_w \nu^T Xw+\lambda||w||_1=0$ if $||X^T\nu||_\infty \leq \lambda$ (by taking $w=0$) and $-\infty$ otherwise. This is because for $||X^T\nu||_\infty \leq \lambda$, $|\nu^T Xw|=|(X^T\nu)^Tw|\leq\lambda||w||_1$.

For $\inf_z \frac{1}{2}||z||_2^2-\nu^T z$, by taking derivative with respect to $z$ and setting it to be 0, we get $z-\nu=0$, which means $z=\nu$. Since the expression is a convex function, $z=\nu$ gives the infimum which equals to $-\frac{||\nu||_2^2}{2}$.

Hence, $g(\nu)=-\frac{||\nu||_2^2}{2}-\nu^T y$ if $||X^T \nu||_\infty\leq\lambda$ and $-\infty$ otherwise. So, the dual problem of LASSO is
\begin{align*}
\max_\nu g(\nu) &\iff \max_\nu -\frac{||\nu||_2^2}{2}-\nu^T y\text{ subject to }||X^T \nu||_\infty\leq\lambda\\
&\iff \min_\nu \frac{||\nu||_2^2}{2}+y^T \nu\text{ subject to }||X^T \nu||_\infty\leq\lambda\\
&\iff \min_v v^TQv+p^Tv\text{ subject to }Av\preceq b
\end{align*}

with $v=\nu\in\mathbb{R}^n$, $Q=\frac{1}{2}I_{n\times n}$, $p=y\in\mathbb{R}^n$, $A=\begin{bmatrix}X^T\\-X^T\end{bmatrix}\in\mathbb{R}^{2d\times n}$, and $b$ is a vector in $\mathbb{R}^{2d}$ with all its elements as $\lambda$.

## Problem 2: Implementation of the barrier method to solve QP

Firstly, consider the central path minimization problem corresponing to the above-mentioned QP. It is $\min_v f(v)=\min_v tf_0(v)+\phi(v)$ with $t>0$, $f_0(v)=v^TQv+p^Tv$ and $\phi(v)=-\sum_{i=1}^{2d} \log(b_i-A^{(i)}v)$, with $A^{(i)}$ denoting the $i$-th row of $A$.

Now, we calculate the gradient and the Hessian of $f(v)$ for the Newton's method by chain rule.
\begin{align*}
\nabla f(v) &= t(2Qv+p) +\sum_{i=1}^{2d} \frac{1}{b_i-A^{(i)}v} {A^{(i)}}^T\\
\nabla^2 f(v) &= 2tQ +\sum_{i=1}^{2d} \left(\frac{1}{b_i-A^{(i)}v}\right)^2 {A^{(i)}}^T A^{(i)}
\end{align*}

With the formulas, we can now implement the functions *centering_step$(Q,p,A,b,t,v0,eps)$*, *barr_method$(Q,p,A,b,v0,eps)$*, and the functions that are used in *centering_step* and *barr_method*.

In [73]:
import numpy as np
import time

In [74]:
def s_feasible(A,b,v):
    # check if the input A,b,v strictly satisfy the constraint
    return ((np.matmul(A,v) >= b).sum() == 0)

In [92]:
def f0(Q,p,v):
    return np.matmul(v.T, np.matmul(Q,v)) + np.dot(p,v)

In [75]:
def f(Q,p,A,b,t,v):
    f0 = np.matmul(v.T, np.matmul(Q,v)) + np.dot(p,v)
    phi = -np.sum(np.log(b-np.matmul(A,v)))
    return t*f0+phi

In [83]:
def backtracking(Q,p,A,b,t,v,alpha,beta,grad,newton_step):
    bt = 1
    too_small = False
    while True:
        if not s_feasible(A,b,v+bt*newton_step):
            bt *= beta
        elif bt < 1e-10:
            too_small = True
            break
        elif f(Q,p,A,b,t,v+bt*newton_step) > f(Q,p,A,b,t,v)+alpha*bt*np.dot(grad,newton_step):
            bt *= beta
        else:
            break
    return bt, too_small

In [106]:
def centering_step(Q,p,A,b,t,v0,eps):
    v = v0
    iter = 0
    bAv_inv = 1/(b-np.matmul(A,v))
    ATA_tensor = np.zeros((A.shape[0], A.shape[1], A.shape[1]))
    for i in range(A.shape[0]):
        ATA_tensor[i,:,:] = np.matmul(A[i,:].T, A[i,:])
    while True:
        # print("centering iter:", iter)
        # compute the Newton step and decrement
        grad = t*(2*np.matmul(Q,v)+p) + np.dot(A.T, bAv_inv)
        hess = 2*t*Q + np.tensordot(bAv_inv**2,ATA_tensor,1)
        newton_step = -np.matmul(np.linalg.inv(hess), grad)
        newton_decrement_sq = np.dot(grad,-newton_step)
        # print("Newton decrement:",newton_decrement_sq)
        
        # stopping criterion
        if newton_decrement_sq <= eps:
            break
        
        # backtracking line search (and stopping criterion for backtracking)
        alpha = 0.1 # parameter of backtracking
        beta = 0.5 # parameter of backtracking
        bt, too_small = backtracking(Q,p,A,b,t,v,alpha,beta,grad,newton_step)
        if too_small:
            break
        # print("bt:", bt)
        # update
        v += bt*newton_step
        iter += 1
        # time.sleep(1)
    return v

In [104]:
def barr_method(Q,p,A,b,v0,t=0.1,mu=1.5,eps=1e-5): # t > 0, mu > 1, eps > 0
    v = v0
    print("initial f0(v)=", f0(Q,p,v0))
    iter = 1
    while True:
        print("barrier iter:", iter)
        # centering step and update
        v = centering_step(Q,p,A,b,t,v0,eps)

        # stopping criterion and increase t
        m = A.shape[0]
        if m/t < eps:
            break
        t *= mu
        iter += 1
        print("f0(v)=", f0(Q,p,v))
        # print("v=", v)
        # time.sleep(3)
    return v

## Problem 3: Testing the implemented functions on randomly generated data, and a comparison on different values of $\mu$

In [107]:
_lambda = 10
# test: w = [3,-2]
# data generation
X = np.array([[1,1],[0.5,-2],[3,0],[-2,1],[-1,5]])
y = np.array([1,5.5,9,-8,-13])

# setting parameters for using the barrier method
(n,d) = X.shape
Q = 0.5*np.eye(n)
p = y
A = np.concatenate((X.T,-X.T), axis=0)
b = _lambda*np.ones(2*d)
# print(A,b)

# initialization and hyperparameter tuning
v0 = np.array([1.,1.,1.,1.,0.])
# t = 
# mu = 
# eps = 

assert s_feasible(A,b,v0)
v = barr_method(Q,p,A,b,v0)
print(f0(Q,p,v))

initial f0(v)= 9.5
barrier iter: 1
f0(v)= -25.13344513565817
barrier iter: 2
f0(v)= -32.606043507864584
barrier iter: 3
f0(v)= -36.114022773549934
barrier iter: 4
f0(v)= -38.991619843943745
barrier iter: 5
f0(v)= -41.18974418045782
barrier iter: 6
f0(v)= -41.890624962325305
barrier iter: 7
f0(v)= -42.369059855377955
barrier iter: 8
f0(v)= -43.0632160521426
barrier iter: 9
f0(v)= -43.318393397588345
barrier iter: 10
f0(v)= -43.647126287739084
barrier iter: 11
f0(v)= -43.83297622273444
barrier iter: 12
f0(v)= -43.902895523845615
barrier iter: 13
f0(v)= -43.97138847192787
barrier iter: 14
f0(v)= -44.0413833001233
barrier iter: 15
f0(v)= -44.053565527904084
barrier iter: 16
f0(v)= -44.078470465714595
barrier iter: 17
f0(v)= -44.10088729507652
barrier iter: 18
f0(v)= -44.110578963531424
barrier iter: 19
f0(v)= -44.11426089339132
barrier iter: 20
f0(v)= -44.118152888460294
barrier iter: 21
f0(v)= -44.12025421902148
barrier iter: 22
f0(v)= -44.12279350862693
barrier iter: 23
f0(v)= -44.124273